# XGBoost Model for Spare Part Demand Forecasting

This notebook implements XGBoost for short-term demand forecasting (1-14 days).

## Objectives
1. Load and prepare data with feature engineering
2. Train XGBoost model
3. Hyperparameter tuning
4. Evaluate model performance
5. Feature importance analysis
6. Compare with Prophet

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded successfully!')
print(f'XGBoost version: {xgb.__version__}')

## 1. Load Data

In [ ]:
# Load daily aggregated demand
df = pd.read_csv('../data/processed/daily_demand.csv', parse_dates=['date'])
print(f'Loaded {len(df)} rows')
df.head()

## 2. Feature Engineering

In [ ]:
# Create date-based features
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_month'] = df['date'].dt.day
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['year'] = df['date'].dt.year
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)

print('Date features created!')
df.head()

In [ ]:
# Create lag features
lags = [1, 7, 14, 30]
for lag in lags:
    df[f'lag_{lag}'] = df['demand_quantity'].shift(lag)

print(f'Lag features created: {lags}')
df.head(35)

In [ ]:
# Create rolling features
windows = [7, 14, 30]
for window in windows:
    df[f'rolling_mean_{window}'] = df['demand_quantity'].shift(1).rolling(window=window).mean()
    df[f'rolling_std_{window}'] = df['demand_quantity'].shift(1).rolling(window=window).std()

print(f'Rolling features created for windows: {windows}')
df.tail()

In [ ]:
# Drop rows with NaN values (from lag and rolling features)
df_clean = df.dropna().copy()
print(f'Rows after dropping NaN: {len(df_clean)} (dropped {len(df) - len(df_clean)})')

In [ ]:
# Check all features
print('Final features:')
print(df_clean.columns.tolist())

## 3. Train-Test Split

In [ ]:
# Define target and features
target = 'demand_quantity'
exclude_cols = ['date', 'demand_quantity', 'revenue']
feature_cols = [col for col in df_clean.columns if col not in exclude_cols]

print(f'Target: {target}')
print(f'Features ({len(feature_cols)}): {feature_cols}')

In [ ]:
# Time series split (no shuffling!)
test_days = 30
train_df = df_clean[:-test_days]
test_df = df_clean[-test_days:]

X_train = train_df[feature_cols]
y_train = train_df[target]
X_test = test_df[feature_cols]
y_test = test_df[target]

print(f'Training set: {X_train.shape}')
print(f'Test set: {X_test.shape}')

## 4. Train XGBoost Model

In [ ]:
# Initialize XGBoost model
model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='reg:squarederror'
)

print('XGBoost model initialized!')

In [ ]:
# Train the model
print('Training XGBoost model...')
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)
print('Model trained successfully!')

## 5. Model Evaluation

In [ ]:
# Generate predictions
y_pred = model.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print('='*50)
print('XGBOOST MODEL EVALUATION METRICS')
print('='*50)
print(f'MAE  (Mean Absolute Error):     {mae:.2f}')
print(f'RMSE (Root Mean Squared Error): {rmse:.2f}')
print(f'MAPE (Mean Absolute % Error):   {mape:.2f}%')
print(f'R2   (R-Squared):               {r2:.4f}')
print('='*50)

In [ ]:
# Visualize actual vs predicted
results_df = pd.DataFrame({
    'Date': test_df['date'].values,
    'Actual': y_test.values,
    'Predicted': y_pred
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df['Date'], y=results_df['Actual'],
                         mode='lines+markers', name='Actual', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=results_df['Date'], y=results_df['Predicted'],
                         mode='lines+markers', name='Predicted', line=dict(color='orange')))

fig.update_layout(title='XGBoost: Actual vs Predicted (Test Period)',
                  xaxis_title='Date', yaxis_title='Demand')
fig.show()

## 6. Feature Importance

In [ ]:
# Get feature importance
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print('Top 10 Features:')
importance_df.head(10)

In [ ]:
# Visualize feature importance
fig = px.bar(importance_df.head(15), x='Importance', y='Feature', orientation='h',
             title='XGBoost Feature Importance (Top 15)',
             color='Importance', color_continuous_scale='Oranges')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

## 7. Hyperparameter Tuning

In [ ]:
# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.15]
}

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=3)

# Grid search
print('Running hyperparameter tuning (this may take a few minutes)...')
grid_search = GridSearchCV(
    xgb.XGBRegressor(random_state=42, objective='reg:squarederror'),
    param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
print('Tuning complete!')

In [ ]:
# Best parameters
print('Best Parameters:')
print(grid_search.best_params_)
print(f'\nBest CV Score (MAE): {-grid_search.best_score_:.2f}')

In [ ]:
# Train final model with best parameters
best_model = grid_search.best_estimator_

# Evaluate on test set
y_pred_best = best_model.predict(X_test)

mae_best = mean_absolute_error(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
mape_best = np.mean(np.abs((y_test - y_pred_best) / y_test)) * 100

print('='*50)
print('TUNED XGBOOST MODEL METRICS')
print('='*50)
print(f'MAE:  {mae_best:.2f} (vs {mae:.2f} before tuning)')
print(f'RMSE: {rmse_best:.2f} (vs {rmse:.2f} before tuning)')
print(f'MAPE: {mape_best:.2f}% (vs {mape:.2f}% before tuning)')
print('='*50)

## 8. Cross-Validation

In [ ]:
# Perform cross-validation on full dataset
tscv = TimeSeriesSplit(n_splits=5)

X_full = df_clean[feature_cols]
y_full = df_clean[target]

cv_scores = cross_val_score(best_model, X_full, y_full, cv=tscv, scoring='neg_mean_absolute_error')

print('Cross-Validation Results (5-fold TimeSeriesSplit):')
print(f'MAE Scores: {-cv_scores}')
print(f'Mean MAE: {-cv_scores.mean():.2f} (+/- {cv_scores.std():.2f})')

## 9. Save Model

In [ ]:
# Save the trained model
import os
os.makedirs('../models', exist_ok=True)

model_data = {
    'model': best_model,
    'feature_names': feature_cols,
    'params': grid_search.best_params_
}

model_path = '../models/xgboost_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model_data, f)

print(f'Model saved to: {model_path}')

In [ ]:
# Save metrics for comparison
metrics = {
    'model': 'XGBoost',
    'mae': mae_best,
    'rmse': rmse_best,
    'mape': mape_best,
    'cv_mae_mean': -cv_scores.mean()
}

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('../models/xgboost_metrics.csv', index=False)
print('Metrics saved!')
metrics_df

## 10. Compare with Prophet

In [ ]:
# Load Prophet metrics if available
try:
    prophet_metrics = pd.read_csv('../models/prophet_metrics.csv')
    xgb_metrics = pd.DataFrame([metrics])
    
    comparison = pd.concat([prophet_metrics, xgb_metrics], ignore_index=True)
    print('Model Comparison:')
    print(comparison.to_string(index=False))
    
    # Determine winner
    if comparison.loc[comparison['model'] == 'XGBoost', 'mae'].values[0] < comparison.loc[comparison['model'] == 'Prophet', 'mae'].values[0]:
        print('\n[WINNER] XGBoost has lower MAE!')
    else:
        print('\n[WINNER] Prophet has lower MAE!')
        
except FileNotFoundError:
    print('Prophet metrics not found. Run Prophet notebook first for comparison.')

In [ ]:
# Visual comparison
try:
    fig = go.Figure(data=[
        go.Bar(name='Prophet', x=['MAE', 'RMSE', 'MAPE'], 
               y=[prophet_metrics['mae'].values[0], prophet_metrics['rmse'].values[0], prophet_metrics['mape'].values[0]],
               marker_color='#3B82F6'),
        go.Bar(name='XGBoost', x=['MAE', 'RMSE', 'MAPE'], 
               y=[mae_best, rmse_best, mape_best],
               marker_color='#F97316')
    ])
    fig.update_layout(title='Model Comparison: Prophet vs XGBoost', barmode='group')
    fig.show()
except:
    print('Could not create comparison chart.')

## Summary

| Metric | Value |
|--------|-------|
| Model | XGBoost |
| Training Period | ~670 days |
| Test Period | 30 days |
| Features | 20+ (lag, rolling, date features) |
| MAE | See above |
| RMSE | See above |
| MAPE | See above |

**Notes:**
- XGBoost uses feature engineering (lags, rolling stats)
- Hyperparameter tuning with GridSearchCV
- Best for short-term forecasts (1-14 days)
- Model saved for deployment

In [ ]:
print('XGBoost Model Training Complete!')